# Sainsbury's API:
## Documentation:
### Base URL:
https://stores.sainsburys.co.uk/api/v1/stores/
### Parameters:
* fields: Not sure what that means. Default value: <code>fields=slfe-list-2.21</code>
* api_client_id: Not sure either. Default value: <code>api_client_id=slfe</code>
* lat: Latitude of search area. Example: <code>lat=51.5073509</code>
* lon: Longitude of search area. Example: <code>lon=-0.1277583</code>
* limit: Stores displayed per page. Default value: <code>limit=25</code>
* store_type: Type of stores displayed. Can contain *main* for superstores and/or *local* for local stores. Example: <code>stores_type=main,local</code>
* sort: How results are sorted. Only found *by_distance* so far. Default value: <code>sort=by_distance</code>
* within: Max distance for which results should be shown. Not sure which unit of measurement is used (miles?). Seems to take a value between 0 and 500. Example: <code>within=15</code>
* page: Page of paginated results. Example: <code>page=1</code>

### Response:
NOTE: Day 0 is monday, and day 6 is Sunday.
Example response:
<pre>{'page_meta': {'limit': 25, 'offset': 0, 'total': 372},
 'results': [{'additional_data': {'main_store_branded_as_local': None,
  'restaurant_branded_as_fresh_kitchen': None},
 'closed': None,
 'code': '4286',
 'contact': {'address1': '36-37 Strand',
  'address2': '',
  'city': 'London',
  'country': 'England',
  'country_code': '',
  'country_name': '',
  'county': '',
  'fax_number': '',
  'manager': 'Debbie Farrow',
  'post_code': 'WC2N 5HY',
  'state': '',
  'telephone': '02079307006'},
 'distance': 0.17963147389184483,
 'district_code': '4286',
 'exception_times': [],
 'location': {'lat': '51.50888', 'lon': '-0.12438'},
 'name': 'Wsmtr The Strand Loc',
 'open': {'current_date': '2020-06-23',
  'current_time': '14:02',
  'is_open': True,
  'open_until': '21:00',
  'today': {'24hrs': False,
   'end_time': '21:00',
   'exception': False,
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  'tomorrow': {'24hrs': False,
   'end_time': '21:00',
   'exception': False,
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]}},
 'opening_times': [{'day': 0,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 1,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 2,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 3,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 4,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 5,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]},
  {'day': 6,
   'end_time': '21:00',
   'start_time': '07:00',
   'times': [{'end_time': '21:00', 'start_time': '07:00'}]}],
 'opening_times_message': None,
 'originator_id': None,
 'other_name': 'Westminster The Strand Local',
 'store_type': 'local'}]
 ...
 } </pre>
 
# Code for retrieving Sainsbury's opening times data:


In [ ]:
def get_sainsburys_data(lat,lng):
    API_URL = "https://stores.sainsburys.co.uk/api/v1/stores/"
    params = {
        'fields': 'slfe-list-2.21',
        'api_client_id': 'slfe',
        'store_type': 'main,local',
        'sort': 'by_distance',
        'within': radius,
        'limit': limit,
        'page': '1',
        'lat': lat,
        'lon': lng,
    }
    rq = requests.get(API_URL, params=params)
    if rq.status_code != 200:
        # print(rq.json())
        return False
    res = rq.json()
    dayKeys = { 0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res[0]['name'] or res[0]['other_name'])
    #e.g.:
    # while True:
    #     if name != res[0]['name'] or res[0]['other_name']:
    #         i += 1
    #     else:
    #         break
    openingHours = res['results'][i]['opening_times']
    hoursArray = []
    for index in range(len(openingHours)):
        key = openingHours[index]['day']
        actualHours = []
        while len(hoursArray) < key:
            #Incase store is closed on a day of the week and therefore not included in the list
            closedDayHours = {  'day' : dayKeys[len(hoursArray)],
                                'open' : False  }
            hoursArray.append(closedDayHours)
        for timeSlotNumber in range(len(openingHours[index]['times'])):
            try:
                actualHoursDict = {}
                actualHoursDict['open'] = openingHours[index]['times'][timeSlotNumber]['start_time']
                actualHoursDict['close'] = openingHours[index]['times'][timeSlotNumber]['end_time']
                actualHours.append(actualHoursDict)
            except:
                #If dictionary keys start_time or end_time dont exist, assume store is closed on that day
                closedDayHours = {  'day' : dayKeys[key],
                                'open' : False  }
                hoursArray.append(closedDayHours)
                break
        keyHours = {'day' : dayKeys[key],
                'open' : True,
                'hours' : actualHours
                }
        hoursArray.append(keyHours)
    while len(hoursArray) < 7:
            #Incase store is closed on a day of the week (at the end of the list/week) and therefore not included in the list
            closedDayHours = {  'day' : dayKeys[len(hoursArray)],
                                'open' : False  }
            hoursArray.append(closedDayHours)
    return hoursArray

# Tesco API

## Parameters:
* limit : number of stores to show (int)
* sort : <code>"near:%22{LATITUDE},{LONGITUDE}%22"</code>
* filter : <code>"category:Store%20AND%20isoCountryCode:x-uk"</code>
* fields : what to return (subset of <code>"name,geo,openingHours,altIds.branchNumber,contact,facilities"</code>)

## Sample headers:
<code>"headers": {
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "traceid": "c78bc741-5b7b-4797-9aa3-bf3a2f96f7a4",
    "x-appkey": "store-locator-web-cde"
  }</code>
  
 ### HOWEVER:
 One appears to be able to bypass any forms of authentication (?) by simply using the following headers:
 
 <code>"headers": {"x-appkey": "store-locator-web-cde"}</code>
 
## Sample API Call directly from website:
<code>rq = requests.get("https://api.tesco.com/tescolocation/v3/locations/search?offset=0&limit=10&sort=near:%2252.62682342529297,1.3366719484329224%22&filter=category:Store%20AND%20isoCountryCode:x-uk&fields=name,geo,openingHours,altIds.branchNumber,contact,facilities", headers={"x-appkey": "store-locator-web-cde"})
</code>
### Sample response (rq.json()['results'][0]):

<pre>{'location': {'id': '2ad4c578-962b-42cd-8541-0f75c1952680', 'name': 'Norwich Plumstead Rd Express', 'altIds': {'branchNumber': 5284}, 'contact': {'address': {'lines': [{'lineNumber': 1, 'text': '197 Plumstead Rd'}], 'town': 'Norwich', 'postcode': 'NR1 4AB'}, 'phoneNumbers': [{'alias': 'Default', 'number': '0345 674 6492'}]}, 'geo': {'coordinates': {'longitude': 1.32715, 'latitude': 52.63697}}, 'openingHours': [{'type': 'Trading', 'standardOpeningHours': {'mo': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'tu': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'we': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'th': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'fr': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'sa': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'su': {'isOpen': 'true', 'open': '0600', 'close': '2300'}}, 'exceptions': [{'date': '2017-12-25', 'hours': {'isOpen': 'false'}}, {'date': '2020-03-21', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2017-12-26', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2020-03-22', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2020-03-18', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-31', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2018-12-24', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2020-03-23', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-25', 'hours': {'isOpen': 'false'}}, {'date': '2020-03-19', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2018-01-01', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2018-12-31', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-26', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2020-01-01', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2018-12-25', 'hours': {'isOpen': 'false'}}, {'date': '2017-12-24', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-24', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2017-12-31', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-01-01', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2020-03-20', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2018-12-26', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}]}], 'facilities': [{'name': 'FREE_FROM', 'tags': ['public', 'food_range'], 'description': 'Free From'}, {'name': 'ATM', 'tags': ['public'], 'description': 'ATM'}, {'name': 'GIFT_CARDS', 'tags': ['public'], 'description': 'Sells Gift Cards'}, {'name': 'TESCO_PAY_PLUS', 'tags': ['public'], 'description': 'Tesco Pay+'}, {'name': 'ASSISTANCE_DOGS', 'tags': ['enabling', 'public'], 'description': 'Assistance dogs are welcome in our store.'}, {'name': 'PAYPOINT', 'tags': ['public'], 'description': 'PayPoint'}]}, 'distanceFrom': {'unit': 'miles', 'value': 0.8}}{'location': {'id': '2ad4c578-962b-42cd-8541-0f75c1952680', 'name': 'Norwich Plumstead Rd Express', 'altIds': {'branchNumber': 5284}, 'contact': {'address': {'lines': [{'lineNumber': 1, 'text': '197 Plumstead Rd'}], 'town': 'Norwich', 'postcode': 'NR1 4AB'}, 'phoneNumbers': [{'alias': 'Default', 'number': '0345 674 6492'}]}, 'geo': {'coordinates': {'longitude': 1.32715, 'latitude': 52.63697}}, 'openingHours': [{'type': 'Trading', 'standardOpeningHours': {'mo': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'tu': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'we': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'th': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'fr': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'sa': {'isOpen': 'true', 'open': '0600', 'close': '2300'}, 'su': {'isOpen': 'true', 'open': '0600', 'close': '2300'}}, 'exceptions': [{'date': '2017-12-25', 'hours': {'isOpen': 'false'}}, {'date': '2020-03-21', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2017-12-26', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2020-03-22', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2020-03-18', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-31', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2018-12-24', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2020-03-23', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-25', 'hours': {'isOpen': 'false'}}, {'date': '2020-03-19', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2018-01-01', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2018-12-31', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-26', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2020-01-01', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2018-12-25', 'hours': {'isOpen': 'false'}}, {'date': '2017-12-24', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-12-24', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2017-12-31', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2019-01-01', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}, {'date': '2020-03-20', 'hours': {'isOpen': 'true', 'open': '0600', 'close': '2200'}}, {'date': '2018-12-26', 'hours': {'isOpen': 'true', 'open': '0800', 'close': '2200'}}]}], 'facilities': [{'name': 'FREE_FROM', 'tags': ['public', 'food_range'], 'description': 'Free From'}, {'name': 'ATM', 'tags': ['public'], 'description': 'ATM'}, {'name': 'GIFT_CARDS', 'tags': ['public'], 'description': 'Sells Gift Cards'}, {'name': 'TESCO_PAY_PLUS', 'tags': ['public'], 'description': 'Tesco Pay+'}, {'name': 'ASSISTANCE_DOGS', 'tags': ['enabling', 'public'], 'description': 'Assistance dogs are welcome in our store.'}, {'name': 'PAYPOINT', 'tags': ['public'], 'description': 'PayPoint'}]}, 'distanceFrom': {'unit': 'miles', 'value': 0.8}}
</pre>

RESULTS:
* of interest is: rq.json()['results'][0]['location']['openingHours'][0]['standardOpeningHours']
* and : rq.json()['results'][0]['location']['openingHours'][0]['exceptions']
* However, exceptions provides you a historical(?) list of dictionaries of all dates that have had exceptions in the past(?), in a seemingly completely unsorted order, making using this information for anything other than extrapolation (**NOT IMPLEMENTED**) of 'special dates'' opening times (e.g. christmas) somewhat unfeasible.



 # Code for retrieving Tesco's opening times data:

In [ ]:
def get_tesco_data(lat, lng):
    API_URL = "https://api.tesco.com/tescolocation/v3/locations/search?offset=0"
    params =    { 'limit' : limit,
                'sort' : "near:%22{0},{1}%22".format(lat,lng),
                'filter' : "category:Store%20AND%20isoCountryCode:x-uk",
                'fields' : "name,geo,openingHours"
                #known fields: "name,geo,openingHours,altIds.branchNumber,contact,facilities"
                }
    headers = {"x-appkey": "store-locator-web-cde"}
    # rq = requests.get(API_URL, params=params, headers=headers)
    #(FOR SOME REASON THIS ^ LINE CAUSES ISSUES SO HAVE REPLACED WITH THE BELOW MANUAL LINE RATHER THAN USING THE PARAMS ARGUMENT OF requests.get())
    rq = requests.get(API_URL + "&limit={0}&sort={1}&filter={2}&fields={3}".format(params['limit'], params['sort'], params['filter'], params['fields']), headers=headers)
    if rq.status_code != 200:
        print(rq.json())
        return False
    res = rq.json()['results']
    i=0
    #INSERT POTENTIAL CHECK THAT STORE MATCHES DESIRED STORENAME!! (res[0]['location']['name'])
    #e.g.:
    # while True:
    #     if name != res[i]['location']['name']:
    #         i += 1
    #     else:
    #         break
    openingHours = res[i]['location']['openingHours'][0]['standardOpeningHours']
    dayKeys = { 'mo' : 'Monday', 'tu' : 'Tuesday', 'we' : 'Wednesday', 'th' : 'Thursday', 'fr' : 'Friday', 'sa' : 'Saturday', 'su' : 'Sunday'}
    hoursArray = []
    for key in openingHours:

        if openingHours[key]['isOpen'] == 'true':
            actualHours = {'open' : openingHours[key]['open'][:2] + ":" + openingHours[key]['open'][2:],
                        'close' : openingHours[key]['close'][:2] + ":" + openingHours[key]['close'][2:]}
            keyHours = {'day' : dayKeys[key],
                    'open' : True,
                    'hours' : [actualHours]
                    }
        else:
            keyHours = {'day' : dayKeys[key],
                    'open' : False}

        hoursArray.append(keyHours)
    return hoursArray